In [1]:
import os

In [2]:
os.getcwd()

'c:\\projects_endtoend\\Chest-X-Ray-Multi-Class-Classifier\\notebook'

In [3]:
os.chdir("../")

In [4]:
os.getcwd() 

'c:\\projects_endtoend\\Chest-X-Ray-Multi-Class-Classifier'

## Update the entity

In [5]:
import tensorflow as tf

import tempfile

In [6]:
model = tf.keras.models.load_model('artifacts/model_training/training.h5')

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    model_path : Path
    training_data : Path
    all_params : dict
    params_image_size : list
    params_batch_size : int
    params_epochs : int

## Update the configuration manager in src config

In [8]:
from cnn_classifier.constants import *
from cnn_classifier.utils.utils import read_yaml, create_directories, save_json

In [9]:
import dagshub
dagshub.init(repo_owner='srinija0208', repo_name='Chest-X-Ray-Multi-Class-Classifier', mlflow=True)

Accessing as srinija0208

Initialized MLflow to track repo "srinija0208/Chest-X-Ray-Multi-Class-Classifier"

Repository srinija0208/Chest-X-Ray-Multi-Class-Classifier initialized!

In [10]:
class ConfigurationManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH, params_file_path=PARAMS_FILE_PATH):

        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root]) 


    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        model_evaluation = ModelEvaluationConfig(
            model_path = "artifacts/model_training/training.h5",
            training_data = "artifacts/data_ingestion",
            all_params = self.params,
            params_image_size = self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE,
            params_epochs = self.params.EPOCHS,
        )

        return model_evaluation

## Update the Components

In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

c:\projects_endtoend\Chest-X-Ray-Multi-Class-Classifier\cnn_env\lib\site-packages\mlflow\utils\requirements_utils.py:12: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [ ]:
class Evaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.model_path)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    # def log_into_mlflow(self):
        
    #     tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
    #     with mlflow.start_run():
    #         mlflow.log_params(self.config.all_params)
    #         mlflow.log_metrics(
    #             {"loss": self.score[0], "accuracy": self.score[1]}
    #         )

    
    #         # Model registry does not work with file store
    #         if tracking_url_type_store != "file":

    #             # Register the model
    #             # There are other ways to use the Model Registry, which depends on the use case,
            
    #             mlflow.keras.log_model(self.model, "model",  registered_model_name="VGG16Model")
    #         else:
    #             mlflow.keras.log_model(self.model, "model")

    def log_into_mlflow(self):
        
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )

            # Manually save the model and log it as an artifact.
            # This avoids the buggy mlflow.keras.log_model() function.
            
            # 1. Use a temporary directory to save the model
            with tempfile.TemporaryDirectory() as tmpdir:
                temp_model_path = os.path.join(tmpdir, "logged_model.h5")
                
                # 2. Force Keras to save in the compatible HDF5 format
                # The model.save() function handles the Path object or string automatically.
                self.model.save(temp_model_path) 
                
                # 3. Log the temporary model file as a model artifact
                mlflow.log_artifact(temp_model_path, artifact_path="model")
                
                if tracking_url_type_store != "file":
                    # Register the model using the artifact location
                    mlflow.register_model(
                        # The URI points to the run ID and the artifact path "model"
                        f"runs:/{mlflow.active_run().info.run_id}/model/logged_model.h5", 
                        "VGG16Model"
                    )

## Update the Pipeline

In [13]:
try:

    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    evaluation = Evaluation(model_evaluation_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

Found 1567 images belonging to 3 classes.


c:\projects_endtoend\Chest-X-Ray-Multi-Class-Classifier\cnn_env\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


25/25 ━━━━━━━━━━━━━━━━━━━━ 77s 3s/step - accuracy: 0.4108 - loss: 2.4253


Registered model 'VGG16Model' already exists. Creating a new version of this model...
2025/12/06 13:28:56 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 12
Created version '12' of model 'VGG16Model'.
